In [1]:
import numpy as np
import pandas as pd
import requests
import subprocess
import time
from os.path import exists
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
import datetime
import matplotlib.pyplot as plt

RAW_FOLDER = 'D:/ibl-website-videos/raw'
PROC_FOLDER = 'D:/ibl-website-videos/proc'
DATA_FOLDER = './data'

# get username/password
with open('../username.txt') as f:
    username = f.readlines()
    username = username[0]
with open('../password.txt') as f:
    password = f.readlines()
    password = password[0]

c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pid = '03c42ea1-1e04-4a3e-9b04-46d8568dcd02'

left_ts = np.load(f'D:/ibl-website-videos/proc/{pid}_left_times.npy')
right_ts = np.load(f'D:/ibl-website-videos/proc/{pid}_right_times.npy')
body_ts = np.load(f'D:/ibl-website-videos/proc/{pid}_body_times.npy')

In [3]:
print((left_ts[0], right_ts[0]))

(19.815951540019817, 19.78198827001978)


In [4]:
print(np.diff(left_ts))
print(np.diff(right_ts))
print(np.diff(body_ts))

[0.01663167 0.016665   0.01663167 ... 0.01663167 0.016665   0.01663167]
[0.01993134 0.00663267 0.006666   ... 0.006666   0.00663267 0.006666  ]
[0.03316335 0.03319668 0.03316335 ... 0.03319668 0.03316335 0.03316335]


In [19]:
left_fr = 1/np.mean(np.diff(left_ts))
right_fr = 1/np.mean(np.diff(right_ts))
body_fr = 1/np.mean(np.diff(body_ts))
framerates = [left_fr, right_fr, body_fr]

In [6]:
np.unique(np.diff(left_ts))

array([0.01663167, 0.01663167, 0.01663167, 0.01663167, 0.01663167,
       0.01663167, 0.01663167, 0.01663167, 0.01663167, 0.01663167,
       0.01663167, 0.016665  , 0.016665  , 0.016665  , 0.016665  ,
       0.016665  , 0.016665  , 0.016665  , 0.016665  , 0.016665  ,
       0.016665  ])

In [7]:
1/0.016665

60.006000600060005

In [8]:
1/0.01663167

60.12625310627255

In [9]:
# Load the first trial time and last trial time
trials = pd.read_csv(f'../{DATA_FOLDER}/{pid}/{pid}_trials.csv')

In [10]:
def ffmpeg_time_format(time_s):
  date = datetime.timedelta(seconds = time_s)
  
  return str(date)

In [11]:
start = trials.start_timestamp.values[0] - 2
end = trials.feedback_timestamp.values[-1] + 2
print((start, end))

start_t = ffmpeg_time_format(start)
length_t = ffmpeg_time_format(end-start)
print((start_t,length_t))

(21.153517750023155, 2742.867119212741)
('0:00:21.153518', '0:45:21.713601')


In [12]:
print(start - left_ts[0])

1.337566210003338


In [20]:
# trim all videos to match the timestamps above
videos = ['left','right','body']
timestamps = [left_ts, right_ts, body_ts]

length_t = ffmpeg_time_format(end-start)

for i, video in enumerate(videos):
  print(f'Running {video}')
  ts = timestamps[i]

  input = f'{RAW_FOLDER}/{pid}_{video}.mp4'
  out = f'D:/ibl-website-videos/proc/test/{video}.mp4'
  
  # check if start is actually before ts[0], that would be bad
  if (ts[0] > start):
    print('First timestamp is after first trial start... bad')
    continue
  
  start_t = ffmpeg_time_format(start - ts[0])

  # call_str = f'ffmpeg -i {input} -ss {start_t} -t {length_t} -vf fps=24,scale=160:128 {out}'
  call = subprocess.call(['ffmpeg',
                        '-r', f'{framerates[i]}',
                        '-i', input,
                        '-ss', start_t,
                        '-t', length_t,
                        '-vf', 'scale=160:128',
                        '-r', '24',
                        out])

Running left
Running right
Running body


In [21]:
# horizontal concat all videos
call = subprocess.call(['ffmpeg',
'-i', f'D:/ibl-website-videos/proc/test/left.mp4',
'-i', f'D:/ibl-website-videos/proc/test/right.mp4',
'-filter_complex', 'hstack',
'D:/ibl-website-videos/proc/test/left_right.mp4'])

call = subprocess.call(['ffmpeg',
'-i', f'D:/ibl-website-videos/proc/test/left_right.mp4',
'-i', f'D:/ibl-website-videos/proc/test/body.mp4',
'-filter_complex', 'hstack',
'D:/ibl-website-videos/proc/test/left_right_body.mp4'])